In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import t
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from scipy.stats import pearsonr
import statsmodels.api as sm
sns.set()

In [81]:
df = pd.read_csv('Data (003).csv')

#Formating a2_300
df_2_300 = df[(df['Scale (L)'] == 300) & (df['Process'] == 2)]
df_2_30 = df[(df['Scale (L)'] == 30) & (df['Process'] == 2)]
a2_300 = pd.DataFrame({'Final g (Delivered)':df_2_300.iloc[:,-17], 'COA':df_2_300.iloc[:,-10]})
a2_300.dropna(inplace=True)
a2_300 = a2_300.apply(pd.to_numeric, errors='coerce')
a2_300.reset_index(drop=True, inplace=True)

#Formating a2_30
a2_30 = pd.DataFrame({'Final g (Delivered)':df_2_30.iloc[:,-17], 'COA':df_2_30.iloc[:,-10]})
a2_30.replace('', pd.NA, inplace=True)
a2_30.drop([150,152], inplace=True)
a2_30 = a2_30.apply(pd.to_numeric, errors='coerce')
a2_30.reset_index(drop=True, inplace=True)
a2_30.dropna(inplace=True)
#a2_30.dropna(inplace=True)


In [95]:
def bootstrap_resample(data_frame, num_bootstrap_samples):
    # Initialize an empty array to store bootstrap samples
    bootstrap_samples = np.empty((num_bootstrap_samples, data_frame.shape[1]))
    
    # Perform bootstrap resampling
    for i in range(num_bootstrap_samples):
        # Randomly sample with replacement from the data array
        random_indices = np.random.choice(data_frame.index, size=data_frame.shape[0], replace=True)
        bootstrap_sample = data_frame.loc[random_indices]
        
        # Compute some statistic on the bootstrap sample (e.g., mean, median, correlation, etc.)
        # For example, to calculate the mean of each column in the bootstrap sample:
        bootstrap_statistic = np.mean(bootstrap_sample, axis=0)
        
        # Store the statistic in the bootstrap_samples array
        bootstrap_samples[i] = bootstrap_statistic
    
    return bootstrap_samples

bootstrap_a2_300 = pd.DataFrame(bootstrap_resample(a2_300, 1000))
bootstrap_a2_300.rename(columns={0: 'Final g (Delivered)', 1: 'COA'}, inplace=True)
bootstrap_a2_30 = pd.DataFrame(bootstrap_resample(a2_30, 1000))
bootstrap_a2_30.rename(columns={0: 'Final g (Delivered)', 1: 'COA '}, inplace=True)

Obtaining Pearson's Correlation and P-Value

In [ ]:
# Compute Pearson's correlation coefficient and p-value for 30 Liter 
corr_coefficient_30, p_value_30 = pearsonr(bootstrap_a2_30['Final g (Delivered)'], bootstrap_a2_30['COA '])

# Print the results
print(f"Pearson's correlation coefficient for 30L: {corr_coefficient_30:.3f}")
print(f"P-value: {p_value_30:.3f}")

In [ ]:
# Compute Pearson's correlation coefficient and p-value for 300 Liter 
corr_coefficient_300, p_value_300 = pearsonr(bootstrap_a2_300['Final g (Delivered)'], bootstrap_a2_300['COA '])

# Print the results
print(f"Pearson's correlation coefficient for 300L: {corr_coefficient_300:.3f}")
print(f"P-value: {p_value_300:.3f}")

Running a Simple Linear Regression

In [92]:
def linear_reg(data_frame, x_column, y_column):
    # Extract the two columns from the DataFrame
    X = data_frame[x_column]
    y = data_frame[y_column]

    # Add a constant term to the predictor variable (X) to fit the intercept
    X = sm.add_constant(X)

    # Fit the linear regression model
    model = sm.OLS(y, X).fit()

    # Get the regression coefficients and other statistics
    print(model.summary())

    # Plot the scatter plot and the fitted regression line
    plt.scatter(X[x_column], y, label='Data')
    plt.plot(X[x_column], model.predict(X), color='red', label='Fitted Line')
    plt.xlabel(x_column)
    plt.ylabel(y_column)
    plt.legend()
    plt.title('Simple Linear Regression')
    plt.show()


300 L

In [ ]:
linear_reg(bootstrap_a2_300, 'Final g (Delivered)', 'COA')

30 L

In [ ]:
linear_reg(bootstrap_a2_30, 'Final g (Delivered)', 'COA')